In [ ]:
import math
import numpy as np
from typing import Any, Callable

class KalmanFilter_MultipleObservation:
    def __init__(self, 
        a:np.ndarray=None, 
        b:np.ndarray=None, 
        c:np.ndarray=None, 
        q:np.ndarray=None, 
        r:np.ndarray=None, 
        init_x:np.ndarray=None, 
        init_p:np.ndarray=None) -> None:
        """線形定常カルマンフィルタ

        複数観測値

        状態変数をn次元ベクトルとする

        モデルのノイズベクトルをp次元ベクトルとする(ややこしい)

        観測値ベクトルをq次元ベクトルとする

        注意として、l次元ベクトルはlx1行列とすること(内部で変換してもいいが、引数のややこしさ回避のためこの様にした)

            > np.array([10, 20]) #間違い

            > np.array([[10],  
                               [20]]) #これ
        
        parameters
        ----------
            a : np.ndarray
                推移行列 
                size : nxn
            b : np.ndarray
                名前がわからない
                size : nxp
            c : np.ndarray 
                観測行列。単一観測値のときと向きが違うことに注意
                size : qxn
            q : np.ndarray
                モデルの共分散
                size : pxp
            r : float
                観測値の共分散
                size : qxq
            init_x : np.ndarray
                状態変数の初期値
                size : nx1
            init_p : np.ndarray
                共分散行列の
                size : nxn"""
        self.a : np.ndarray = a
        self.b : np.ndarray = b
        self.c : np.ndarray = c
        self.x : np.ndarray = init_x
        self.p : np.ndarray = init_p
        self.q : np.ndarray = q
        self.r : np.ndarray = r
    @property
    def observation(self) -> float:
        """Cx"""
        return self.c@self.x
    def update(self, y) -> None:
        """アップデート"""
        #予測
        x_ = self.a@self.x
        p_ = self.a@self.p@self.a.T+self.b@self.q@self.b.T
        #修正（フィルタリング）
        self.g = p_@self.c.T@np.linalg.solve(self.c@p_@self.c.T+self.r, np.eye(self.r.shape[0]))
        self.x = x_+self.g@(y-self.c@x_)
        self.p = (np.eye(len(self.x))-self.g@self.c)@p_
